# StockVision - 新鮮な株価データ取得 (2024年1月〜現在)

このNotebookは正確な日本株価データを取得し、StockVisionシステムにインポート可能なCSVファイルを作成します。

## 取得期間
- **開始**: 2024年1月1日
- **終了**: 現在
- **対象**: 主要日本株10銘柄

## 改善点
- より長期間のデータ
- データ品質チェック
- 異常値検出
- 正確な価格情報

In [ ]:
# 必要なライブラリのインストール
!pip install yfinance pandas numpy matplotlib seaborn

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("ライブラリ読み込み完了")
print(f"現在時刻: {datetime.now()}")

In [ ]:
# 主要日本株銘柄リスト
stocks = {
    '7203.T': 'Toyota Motor Corporation',
    '9984.T': 'SoftBank Group Corp.',
    '6758.T': 'Sony Group Corporation',
    '7974.T': 'Nintendo Co., Ltd.',
    '6861.T': 'Keyence Corporation',
    '9983.T': 'Fast Retailing Co., Ltd.',
    '4689.T': 'Yahoo Japan Corporation',
    '6367.T': 'Daikin Industries, Ltd.',
    '4502.T': 'Takeda Pharmaceutical Company Limited',
    '8267.T': 'AEON Co., Ltd.'
}

print(f"対象銘柄数: {len(stocks)}")
for code, name in stocks.items():
    print(f"  {code}: {name}")

In [ ]:
# データ取得期間の設定
start_date = '2024-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

print("データ取得期間:")
print(f"  開始日: {start_date}")
print(f"  終了日: {end_date}")

# 期間の日数計算
start_dt = datetime.strptime(start_date, '%Y-%m-%d')
end_dt = datetime.strptime(end_date, '%Y-%m-%d')
days_diff = (end_dt - start_dt).days
print(f"  期間: {days_diff}日間")

In [ ]:
# 株価データ取得関数
def fetch_stock_data(symbol, start, end, company_name):
    """指定された銘柄の株価データを取得"""
    try:
        print(f"取得中: {symbol} ({company_name})")
        
        # yfinanceでデータ取得
        ticker = yf.Ticker(symbol)
        hist = ticker.history(start=start, end=end)
        
        if hist.empty:
            print(f"  ⚠️ データが見つかりません: {symbol}")
            return None
        
        # データフレーム整形
        df = hist.reset_index()
        df['Stock_Code'] = symbol.replace('.T', '')  # .Tを除去
        df['Company_Name'] = company_name
        
        # カラム名を統一
        df = df.rename(columns={
            'Date': 'Date',
            'Open': 'Open',
            'High': 'High', 
            'Low': 'Low',
            'Close': 'Close',
            'Volume': 'Volume'
        })
        
        # 必要なカラムのみ選択
        df = df[['Stock_Code', 'Company_Name', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
        
        # データ品質チェック
        null_count = df.isnull().sum().sum()
        if null_count > 0:
            print(f"  ⚠️ 欠損値あり: {null_count}件")
            df = df.dropna()  # 欠損値削除
        
        # 価格の妥当性チェック
        min_price = df[['Open', 'High', 'Low', 'Close']].min().min()
        max_price = df[['Open', 'High', 'Low', 'Close']].max().max()
        
        if min_price <= 0:
            print(f"  ⚠️ 異常な価格データあり (最小値: {min_price})")
            df = df[(df[['Open', 'High', 'Low', 'Close']] > 0).all(axis=1)]
        
        print(f"  ✅ 取得成功: {len(df)}件 (価格範囲: {min_price:.2f}〜{max_price:.2f}円)")
        return df
        
    except Exception as e:
        print(f"  ❌ エラー: {symbol} - {str(e)}")
        return None

# 全銘柄のデータを取得
all_data = []
success_count = 0
total_records = 0

print("\n=== 株価データ取得開始 ===")
for symbol, company_name in stocks.items():
    data = fetch_stock_data(symbol, start_date, end_date, company_name)
    if data is not None:
        all_data.append(data)
        success_count += 1
        total_records += len(data)

print("\n=== 取得結果 ===")
print(f"成功: {success_count}/{len(stocks)} 銘柄")
print(f"総レコード数: {total_records:,}件")

In [ ]:
# データの結合と検証
if all_data:
    # 全データを結合
    combined_df = pd.concat(all_data, ignore_index=True)
    
    print("=== データ検証 ===")
    print(f"総レコード数: {len(combined_df):,}")
    print(f"銘柄数: {combined_df['Stock_Code'].nunique()}")
    print(f"期間: {combined_df['Date'].min()} 〜 {combined_df['Date'].max()}")
    
    # 各銘柄の統計
    print("\n=== 銘柄別データ数 ===")
    stock_counts = combined_df.groupby(['Stock_Code', 'Company_Name']).size().reset_index(name='Records')
    for _, row in stock_counts.iterrows():
        print(f"  {row['Stock_Code']}: {row['Records']:,}件 ({row['Company_Name']})")
    
    # データ品質チェック
    print("\n=== データ品質チェック ===")
    
    # 重複チェック
    duplicates = combined_df.duplicated(['Stock_Code', 'Date']).sum()
    print(f"重複レコード: {duplicates}件")
    
    # 価格異常値チェック
    for stock_code in combined_df['Stock_Code'].unique():
        stock_data = combined_df[combined_df['Stock_Code'] == stock_code]
        
        # 価格の統計
        close_prices = stock_data['Close']
        mean_price = close_prices.mean()
        std_price = close_prices.std()
        
        # 3σを超える異常値検出
        anomalies = stock_data[
            (stock_data['Close'] < mean_price - 3 * std_price) |
            (stock_data['Close'] > mean_price + 3 * std_price)
        ]
        
        if len(anomalies) > 0:
            print(f"  {stock_code}: {len(anomalies)}件の異常値候補")
            for _, anomaly in anomalies.iterrows():
                z_score = (anomaly['Close'] - mean_price) / std_price
                print(f"    {anomaly['Date'].strftime('%Y-%m-%d')}: {anomaly['Close']:.2f}円 (Z={z_score:.2f})")
        else:
            print(f"  {stock_code}: 異常値なし (平均: {mean_price:.2f}円)")
    
    print("\n✅ データ検証完了")
else:
    print("❌ データが取得できませんでした")

In [ ]:
# データ可視化
if all_data:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('株価データ概要 (2024年1月〜現在)', fontsize=16)
    
    # 1. 各銘柄の終値推移
    ax1 = axes[0, 0]
    for stock_code in combined_df['Stock_Code'].unique()[:5]:  # 上位5銘柄
        stock_data = combined_df[combined_df['Stock_Code'] == stock_code]
        ax1.plot(pd.to_datetime(stock_data['Date']), stock_data['Close'], 
                label=stock_code, alpha=0.7)
    ax1.set_title('終値推移 (主要5銘柄)')
    ax1.set_xlabel('日付')
    ax1.set_ylabel('価格 (円)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. 日次データ数の分布
    ax2 = axes[0, 1]
    daily_counts = combined_df.groupby('Date').size()
    ax2.hist(daily_counts, bins=20, alpha=0.7, color='skyblue')
    ax2.set_title('日別データ数分布')
    ax2.set_xlabel('1日あたりのデータ数')
    ax2.set_ylabel('日数')
    ax2.grid(True, alpha=0.3)
    
    # 3. 銘柄別データ数
    ax3 = axes[1, 0]
    stock_counts = combined_df['Stock_Code'].value_counts()
    stock_counts.plot(kind='bar', ax=ax3, color='lightgreen')
    ax3.set_title('銘柄別データ数')
    ax3.set_xlabel('銘柄コード')
    ax3.set_ylabel('データ数')
    ax3.tick_params(axis='x', rotation=45)
    ax3.grid(True, alpha=0.3)
    
    # 4. 出来高分布
    ax4 = axes[1, 1]
    ax4.hist(np.log10(combined_df['Volume'] + 1), bins=30, alpha=0.7, color='orange')
    ax4.set_title('出来高分布 (log10)')
    ax4.set_xlabel('log10(出来高)')
    ax4.set_ylabel('頻度')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 データ可視化完了")

In [ ]:
# CSVファイル出力
if all_data:
    # ファイル名を現在日時で作成
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'fresh_stock_data_2024_{timestamp}.csv'
    
    # CSV出力
    combined_df.to_csv(filename, index=False, encoding='utf-8')
    
    print("=== CSV出力完了 ===")
    print(f"ファイル名: {filename}")
    print(f"ファイルサイズ: {len(combined_df):,} レコード")
    print(f"カラム: {list(combined_df.columns)}")
    
    # サンプルデータ表示
    print("\n=== サンプルデータ (最初の5行) ===")
    print(combined_df.head())
    
    print("\n✅ 処理完了！")
    print("\n📋 次のステップ:")
    print("1. CSVファイルをGoogle Driveにアップロード")
    print("2. 共有リンクを取得")
    print("3. StockVisionでインポート実行")
    
else:
    print("❌ CSVの出力に失敗しました")

In [ ]:
# ファイルダウンロード (Google Colab)
try:
    from google.colab import files
    if 'filename' in locals():
        files.download(filename)
        print(f"📥 {filename} をダウンロードしました")
    else:
        print("❌ ダウンロードするファイルがありません")
except ImportError:
    print("ℹ️ Google Colab環境ではありません（ローカル環境）")
    print(f"ファイルは現在のディレクトリに保存されました: {filename if 'filename' in locals() else '未作成'}")